In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
%cd "/content/drive/MyDrive/FastAPI Demo"

/content/drive/MyDrive/FastAPI Demo


In [ ]:
import os

In [2]:
from models.unet_model import unet
from glob import glob
from process.pre_process import resize_image, new_resize_image, split_image, merge_images
import os
import json
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import base64
import numpy as np
import io
import math
import zipfile

In [4]:
zip_path = "OneDrive_2024-11-25 (1).zip"
extract_path = "images"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [5]:
def cut_unecessary_img(image):
    # Check if the image is valid
    if image is None:
        print("Invalid image.")
        return image

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Set the threshold value and threshold the image
    threshold_value = 185
    ret, thresholded_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)

    # Invert the thresholded image
    thresholded_image = cv2.bitwise_not(thresholded_image)

    # Find all contours in the thresholded image
    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Create a black mask with the same size as the grayscale image
    mask = np.zeros_like(gray_image)

    # Save contours that meet the condition into a list
    new_contours = []

    # Set the minimum height for contours (50% of the image height)
    MIN_HEIGHT = image.shape[1] * 0.5

    # Filter contours with height greater than or equal to MIN_HEIGHT
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if h >= MIN_HEIGHT:
            new_contours.append(cnt)

    # If no suitable contour is found, return the original image
    if not new_contours:
        return image

    # Get the largest contour that meets the condition
    con = new_contours[0]
    x, y, w, h = cv2.boundingRect(con)
    if h < image.shape[0] and w < image.shape[1]:
        # Draw a white contour on the mask
        cv2.drawContours(mask, [con], -1, (255), thickness=cv2.FILLED)

        # Apply the mask to the original image to keep the white contour area
        result = cv2.bitwise_and(image, image, mask=mask)

        # Crop the image
        result = result[y:y+h, x:x+w]

    result = result.astype(np.uint8)
    return result


In [6]:
def get_contours_max(image):

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Set the threshold value and threshold the image
    threshold_value = 185
    ret, thresholded_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)

    # Invert the thresholded image
    thresholded_image = cv2.bitwise_not(thresholded_image)

    # Find all contours in the thresholded image
    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Create a black mask with the same size as the grayscale image
    mask = np.zeros_like(gray_image)

    # Save contours that meet the condition into a list
    new_contours = []

    # Set the minimum height for contours (50% of the image height)
    MIN_HEIGHT = image.shape[1] * 0.5

    # Filter contours with height greater than or equal to MIN_HEIGHT
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if h >= MIN_HEIGHT:
            new_contours.append(cnt)

    # If no suitable contour is found, return the original image
    if not new_contours:
        return image

    # Get the largest contour that meets the condition
    con = new_contours[0]
    cv2.drawContours(mask, [con], -1, (255), 2)
    plt.imshow(mask)
    plt.axis('off')
    plt.show()
    return con

In [16]:
train_path = glob(os.path.join("images","01. Train 1","01. Tư liệu AI-gốc","*"))
train_path

['images/01. Train 1/01. Tư liệu AI-gốc/F1_B2_6h_12.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/F1_B2_24h_04.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/E1_B2_2h_09.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/E1_B2_6h_10.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/D1-B1-6H-11.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/D1_B1_0h_02.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/D1-B1-20H-08.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/E1_B2_6h_06.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/E1_B2_4h_02.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/F1_B2_6h_04.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/E1_B1_0h_05.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/F1_B2_24h_05.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/D1-B1-8H-06.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/D1-B1-2H-06.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/F1_B2_4h_02.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/D1-B1-4H-03.jpeg',
 'images/01. Train 1/01. Tư liệu AI-gốc/E1_B2_2h_02.j

In [17]:
len(train_path)

52

In [18]:
def predict_mask(image):
        image_normalize = image.astype(np.float32) / 255.0

        # Split image into patches
        image_array = split_image(image_normalize)

        # Predict using U-Net model
        predictions = []
        batch_size = 1

        for i in range(0, len(image_array), batch_size):
            batch = np.array(image_array[i:i+batch_size])
            batch_predictions = unet.predict(batch)
            predictions.extend(batch_predictions)

        predictions = np.array(predictions)
        merge_mask = merge_images(image,predictions)
        merge_mask = (merge_mask > 0.5).astype(np.uint8) * 255

        return merge_mask

In [19]:
def resize_image(image, value=0):
    """
    Resize the input image by adding padding evenly on all sides to match the nearest multiple of 256 for both dimensions.

    Parameters:
    image (array): The input image in BGR format.
    value (int or tuple): The padding value (default is 0 for black).

    Returns:
    array: The resized image with padding added evenly on all sides.
    """
    height, width, _ = image.shape

    # Tính kích thước mục tiêu
    target_height = ((height + 255) // 256) * 256
    target_width = ((width + 255) // 256) * 256

    # Tính lượng padding cần thêm
    pad_height = target_height - height
    pad_width = target_width - width

    # Phân bổ padding đều trên/dưới và trái/phải
    top_pad = pad_height // 2
    bottom_pad = pad_height - top_pad
    left_pad = pad_width // 2
    right_pad = pad_width - left_pad

    # Thêm padding cho ảnh
    padded_image = cv2.copyMakeBorder(image, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=value)

    return padded_image


In [20]:
def get_contours_max(image):

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Set the threshold value and threshold the image
    threshold_value = 185
    ret, thresholded_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)

    # Invert the thresholded image
    thresholded_image = cv2.bitwise_not(thresholded_image)

    # Find all contours in the thresholded image
    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Create a black mask with the same size as the grayscale image
    mask = np.zeros_like(gray_image)

    # Save contours that meet the condition into a list
    new_contours = []

    # Set the minimum height for contours (50% of the image height)
    MIN_HEIGHT = image.shape[1] * 0.5

    # Filter contours with height greater than or equal to MIN_HEIGHT
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if h >= MIN_HEIGHT:
            new_contours.append(cnt)

    # If no suitable contour is found, return the original image
    if not new_contours:
        return image

    # Get the largest contour that meets the condition
    con = new_contours[0]
    cv2.drawContours(mask, [con], -1, (255), 2)
    plt.imshow(mask)
    plt.axis('off')
    plt.show()
    return con

In [24]:
images_train = []
for i in train_path:
    print(os.path.basename(i))
    img = cv2.imread(i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cut_unecessary_img(img)
    img = resize_image(img, img[0][0].tolist())
    images_train.append(img)

F1_B2_6h_12.jpeg
F1_B2_24h_04.jpeg
E1_B2_2h_09.jpeg
E1_B2_6h_10.jpeg
D1-B1-6H-11.jpeg
D1_B1_0h_02.jpeg
D1-B1-20H-08.jpeg
E1_B2_6h_06.jpeg
E1_B2_4h_02.jpeg
F1_B2_6h_04.jpeg
E1_B1_0h_05.jpeg
F1_B2_24h_05.jpeg
D1-B1-8H-06.jpeg
D1-B1-2H-06.jpeg
F1_B2_4h_02.jpeg
D1-B1-4H-03.jpeg
E1_B2_2h_02.jpeg
F1_B2_2h_02.jpeg
E1_B2_6h_03.jpeg
F1_B2_20h_07.jpeg
F1_B2_10h_03.jpeg
F1_B2_20h_09.jpeg
D1-B1-12H-02.jpeg
F1_B2_2h_09.jpeg
D1-B1-6H-02.jpeg
D1-B1-20H-01.jpeg
D1-B1-12H-04.jpeg
F1_B2_0h_12.jpeg
F1_B2_4h_12.jpeg
D1-B1-4H-08.jpeg
D1-B1-4H-05.jpeg
D1-B1-2H-04.jpeg
E1_B2_4h_04.jpeg
F1_B2_24h_03.jpeg
D1-B1-8H-08.jpeg
D1_B1_0h_12.jpeg
F1_B2_0h_11.jpeg
F1_B2_10h_06.jpeg
D1-B1-10H-11.jpeg
F1_B2_0h_08.jpeg
D1-B1-12H-11.jpeg
D1-B1-8H-12.jpeg
F1_B2_6h_01.jpeg
F1_B2_4h_06.jpeg
E1_B1_0h_02.jpeg
E1_B1_0h_11.jpeg
E1_B2_2h_11.jpeg
F1_B2_20h_12.jpeg
F1_B2_10h_08.jpeg
D1-B1-2H-01.jpeg
E1_B2_4h_09.jpeg
D1-B1-20H-06.jpeg


In [26]:
!nvidia-smi

Mon Nov 25 14:13:19 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off |   00000000:01:00.0 Off |                  N/A |
|  0%   48C    P8             23W /  260W |    9958MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [28]:
mask_train = []
for img in images_train:
    mask = predict_mask(img)
    mask_train.append(mask)
    plt.imshow(mask, cmap = 'gray')
    plt.axis('off')
    plt.show()

E0000 00:00:1732518815.595958 2555381 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1732518815.600902 2555381 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2024-11-25 14:13:35.603257: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at xla_ops.cc:577 : FAILED_PRECONDITION: DNN library initialization failed. Look at the er

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/media/data3/users/nhatthang/FastAPIDemo/yeast_env/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/media/data3/users/nhatthang/FastAPIDemo/yeast_env/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/media/data3/users/nhatthang/FastAPIDemo/yeast_env/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/media/data3/users/nhatthang/FastAPIDemo/yeast_env/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line

In [45]:
def save_crop_mask_box(mask, contours_max, image_name):
    os.makedirs(os.path.join("saved_cell_2511", image_name), exist_ok=True)
    MIN_HEIGHT = 12
    MAX_HEIGHT = mask.shape[0] * 0.25
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # plt.figure(figsize=(10, 5))
    # plt.subplot(1, 2, 1)
    # plt.imshow(mask)
    i = 1
    for cnt in contours:
        in_side = 1
        cnt = np.array(cnt)
        cnt = cnt.reshape(-1,2)
        x, y, w, h = cv2.boundingRect(cnt)
        y1 = max(0, y - 1)
        y2 = min(mask.shape[0], y + h + 1)
        x1 = max(0, x - 1)
        x2 = min(mask.shape[1], x + w + 1)
        point = [(x1,y1), (x1,y2), (x2,y1), (x2,y2)]
        for point_tuple in point:
            # Convert point coordinates to the correct data type (tuple of floats)
            point_tuple = (float(point_tuple[0]), float(point_tuple[1]))
            if(cv2.pointPolygonTest(contours_max, point_tuple, False) < 0):
                in_side = 0
                break
        if in_side == 0:
            continue
        x, y, w, h = cv2.boundingRect(cnt)
        if w >= MIN_HEIGHT and h < MAX_HEIGHT and h > MIN_HEIGHT:
            x1 = max(0, x - 6)
            y1 = max(0, y - 6)
            x2 = min(mask.shape[1], x + w + 6)
            y2 = min(mask.shape[0], y + h + 6)
            crop_number = mask[y1:y2, x1:x2]
            cv2.imwrite(os.path.join("saved_cell_2511", image_name, f"{image_name}_{i}_mask.png"), crop_number)
            i += 1
            # plt.imshow(crop_number)
            # plt.axis('off')
            # plt.show()
    print(i)
    print("-" * 100)

In [46]:
def save_crop_image_without_bounding_box(image, mask, contours_max, image_name):
    os.makedirs(os.path.join("saved_cell_2511", image_name), exist_ok=True)
    MIN_HEIGHT = 12
    MAX_HEIGHT = image.shape[0] * 0.25
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # plt.figure(figsize=(10, 5))
    # plt.subplot(1, 2, 1)
    # plt.imshow(image)
    i = 1
    for cnt in contours:
        in_side = 1
        cnt = np.array(cnt)
        cnt = cnt.reshape(-1,2)
        x, y, w, h = cv2.boundingRect(cnt)
        y1 = max(0, y - 1)
        y2 = min(mask.shape[0], y + h + 1)
        x1 = max(0, x - 1)
        x2 = min(mask.shape[1], x + w + 1)
        point = [(x1,y1), (x1,y2), (x2,y1), (x2,y2)]
        for point_tuple in point:
            # Convert point coordinates to the correct data type (tuple of floats)
            point_tuple = (float(point_tuple[0]), float(point_tuple[1]))
            if(cv2.pointPolygonTest(contours_max, point_tuple, False) < 0):
                in_side = 0
                break
        if in_side == 0:
            continue
        if w >= MIN_HEIGHT and h < MAX_HEIGHT and h > MIN_HEIGHT:
            x1 = max(0, x - 6)
            y1 = max(0, y - 6)
            x2 = min(image.shape[1], x + w + 6)
            y2 = min(image.shape[0], y + h + 6)
            crop_number = image[y1:y2, x1:x2]
            cv2.imwrite(os.path.join("saved_cell_2511", image_name, f"{image_name}_{i}.png"), crop_number)
            i += 1
            # plt.imshow(crop_number)
            # plt.axis('off')
            # plt.show()
    print(i)
    print("-" * 100)

In [47]:
def save_crop_image_with_bounding_box(image, mask, contours_max, image_name):
    # os.makedirs(os.path.join("images", "saved_cell_with_contours", image_name), exist_ok=True)
    MIN_HEIGHT = 12
    MAX_HEIGHT = image.shape[0] * 0.25
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # plt.figure(figsize=(10, 5))
    # plt.subplot(1, 2, 1)
    # plt.imshow(image)
    i = 1
    for cnt in contours:
        in_side = 1
        cnt = np.array(cnt)
        cnt = cnt.reshape(-1,2)
        x, y, w, h = cv2.boundingRect(cnt)
        y1 = max(0, y - 1)
        y2 = min(mask.shape[0], y + h + 1)
        x1 = max(0, x - 1)
        x2 = min(mask.shape[1], x + w + 1)
        point = [(x1,y1), (x1,y2), (x2,y1), (x2,y2)]
        for point_tuple in point:
            # Convert point coordinates to the correct data type (tuple of floats)
            point_tuple = (float(point_tuple[0]), float(point_tuple[1]))
            if(cv2.pointPolygonTest(contours_max, point_tuple, False) < 0):
                in_side = 0
                break
        if in_side == 0:
            continue
        x, y, w, h = cv2.boundingRect(cnt)
        if w >= MIN_HEIGHT and h < MAX_HEIGHT and h > MIN_HEIGHT:
            cv2.drawContours(image, [cnt], -1, (0, 0, 255), 1)
            x1 = max(0, x - 6)
            y1 = max(0, y - 6)
            x2 = min(image.shape[1], x + w + 6)
            y2 = min(image.shape[0], y + h + 6)
            crop_number = image[y1:y2, x1:x2]
            cv2.imwrite(os.path.join("saved_cell_2511", image_name, f"{image_name}_{i}_outline.png"), crop_number)
            i += 1
            # plt.imshow(crop_number)
            # plt.axis('off')
            # plt.show()
    print(i)
    print("-" * 100)

In [48]:
images_name = [os.path.basename(i).split('.')[0] for i in train_path]
images_name

['D1_B1_0h_12',
 'D1_B1_0h_02',
 'D1-B1-20H-06',
 'D1-B1-20H-08',
 'D1-B1-20H-01',
 'D1-B1-12H-04',
 'D1-B1-10H-11',
 'D1-B1-2H-04',
 'D1-B1-12H-11',
 'D1-B1-2H-01',
 'D1-B1-12H-02',
 'D1-B1-2H-06',
 'D1-B1-4H-03',
 'D1-B1-6H-02',
 'D1-B1-4H-05',
 'D1-B1-4H-08',
 'D1-B1-8H-08',
 'D1-B1-8H-06',
 'D1-B1-6H-11',
 'E1_B1_0h_02',
 'D1-B1-8H-12',
 'E1_B1_0h_05',
 'E1_B1_0h_11',
 'E1_B2_2h_02',
 'E1_B2_4h_02',
 'E1_B2_2h_09',
 'E1_B2_2h_11',
 'E1_B2_4h_04',
 'E1_B2_4h_09',
 'E1_B2_6h_03',
 'E1_B2_6h_06',
 'E1_B2_6h_10',
 'F1_B2_0h_08',
 'F1_B2_0h_12',
 'F1_B2_0h_11',
 'F1_B2_10h_03',
 'F1_B2_10h_06',
 'F1_B2_10h_08',
 'F1_B2_20h_09',
 'F1_B2_20h_07',
 'F1_B2_20h_12',
 'F1_B2_24h_03',
 'F1_B2_24h_04',
 'F1_B2_24h_05',
 'F1_B2_2h_09',
 'F1_B2_2h_02',
 'F1_B2_4h_02',
 'F1_B2_4h_06',
 'F1_B2_6h_01',
 'F1_B2_4h_12',
 'F1_B2_6h_04',
 'F1_B2_6h_12',
 'F1_B2_7']

In [49]:
for img, mk, ctm, img_name in zip(images_train, mask_train, contours_max, images_name):
    save_crop_mask_box(mk, ctm, img_name)
    save_crop_image_without_bounding_box(img, mk, ctm, img_name)
    save_crop_image_with_bounding_box(img, mk, ctm, img_name)

33
----------------------------------------------------------------------------------------------------
33
----------------------------------------------------------------------------------------------------
33
----------------------------------------------------------------------------------------------------
53
----------------------------------------------------------------------------------------------------
53
----------------------------------------------------------------------------------------------------
53
----------------------------------------------------------------------------------------------------
130
----------------------------------------------------------------------------------------------------
130
----------------------------------------------------------------------------------------------------
130
----------------------------------------------------------------------------------------------------
120
---------------------------------------------------------